In [1]:
import pandas as pd
import sys
sys.path.append('../src') 
from data_loader import ChestXray8Dataset
import time
import torch.optim as optim
import torch.nn as nn

import os
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm # Progession bar

# Configuration settings
data_dir = Path.cwd().parent / 'data'
images_dir = data_dir / 'images'
metadata_dir = data_dir /'metadata'/ 'Data_Entry_2017_v2020.csv'
train_list_path = data_dir /'metadata'/ 'train_val_list.txt'
test_list_path = data_dir / 'metadata' /'test_list.txt'
transform = transforms.Compose([
    transforms.Resize((224, 224)),             # Redimensionar a 224x224
    transforms.ToTensor(),                     # Convertir a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalización
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
train_dataset = ChestXray8Dataset(
    img_dir=images_dir, 
    metadata_file=metadata_dir, 
    split_file=train_list_path,
    mode='train',  # Training mode
    transform=transform
)
test_dataset = ChestXray8Dataset(
    img_dir=images_dir, 
    metadata_file=metadata_dir, 
    split_file=test_list_path,
    mode='test',  # Training mode
    transform=transform
)
train_loader =  DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Modelo DEC

In [13]:
class FeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(FeatureExtractor, self).__init__()
        resnet = models.resnet50(pretrained=pretrained)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Quitar capa final

    def forward(self, x):
        return self.features(x).view(x.size(0), -1)  # Aplanar las características

import torch.nn as nn

class DEC(nn.Module):
    def __init__(self, feature_extractor, n_labels=14, alpha=1.0):
        super(DEC, self).__init__()
        self.feature_extractor = feature_extractor
        self.n_labels = n_labels
        self.alpha = alpha

        # Cálculo dinámico de la dimensión de salida
        dummy_input = torch.zeros(1, 3, 224, 224).cuda()  # Mover dummy_input a GPU
        with torch.no_grad():
            output = self.feature_extractor(dummy_input)
            feature_dim = output.shape[1]

        self.centroids = nn.Parameter(torch.randn(n_labels, feature_dim).cuda())  # Centroides en GPU

    def forward(self, x):
        features = self.feature_extractor(x)
        distances = torch.cdist(features, self.centroids)
        probabilities = torch.sigmoid(-distances)
        return probabilities


## Entrenamiento

In [14]:
import time
from sklearn.metrics import hamming_loss, f1_score
import pandas as pd

def train_dec_multilabel(dataloader, model, optimizer, n_epochs=20, output_dir="output"):
    os.makedirs(output_dir, exist_ok=True)
    metrics_file = os.path.join(output_dir, "metrics.csv")
    model_file = os.path.join(output_dir, "dec_model_multilabel.pth")

    metrics = []
    start_time = time.time()

    model.train()
    for epoch in range(n_epochs):
        total_loss = 0
        total_hamming = 0
        total_f1_macro = 0
        total_f1_micro = 0
        total_samples = 0

        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()

            optimizer.zero_grad()

            probabilities = model(inputs)
            loss = model.loss(probabilities, labels)
            loss.backward()
            optimizer.step()

            predictions = (probabilities > 0.5).float()
            hamming = hamming_loss(labels.cpu().numpy(), predictions.cpu().numpy())
            f1_macro = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average="macro")
            f1_micro = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average="micro")

            total_loss += loss.item()
            total_hamming += hamming * len(inputs)
            total_f1_macro += f1_macro * len(inputs)
            total_f1_micro += f1_micro * len(inputs)
            total_samples += len(inputs)

        epoch_loss = total_loss / len(dataloader)
        epoch_hamming = total_hamming / total_samples
        epoch_f1_macro = total_f1_macro / total_samples
        epoch_f1_micro = total_f1_micro / total_samples

        metrics.append({
            "Epoch": epoch + 1,
            "Loss": epoch_loss,
            "Hamming Loss": epoch_hamming,
            "F1 Macro": epoch_f1_macro,
            "F1 Micro": epoch_f1_micro
        })

        print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {epoch_loss:.4f}, "
              f"Hamming Loss: {epoch_hamming:.4f}, F1 Macro: {epoch_f1_macro:.4f}, "
              f"F1 Micro: {epoch_f1_micro:.4f}")

    pd.DataFrame(metrics).to_csv(metrics_file, index=False)
    torch.save(model.state_dict(), model_file)
    print(f"Entrenamiento completado en {time.time() - start_time:.2f} segundos. "
          f"Modelo y métricas guardados en {output_dir}.")


## Evaluar

In [15]:
def evaluate_dec_multilabel(model, dataloader):
    model.eval()
    total_hamming = 0
    total_f1_macro = 0
    total_f1_micro = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.cuda()
            labels = labels.cuda()

            probabilities = model(inputs)
            predictions = (probabilities > 0.5).float()

            hamming = hamming_loss(labels.cpu().numpy(), predictions.cpu().numpy())
            f1_macro = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average="macro")
            f1_micro = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average="micro")

            total_hamming += hamming * len(inputs)
            total_f1_macro += f1_macro * len(inputs)
            total_f1_micro += f1_micro * len(inputs)
            total_samples += len(inputs)

    return {
        "Hamming Loss": total_hamming / total_samples,
        "F1 Macro": total_f1_macro / total_samples,
        "F1 Micro": total_f1_micro / total_samples
    }


In [16]:

# Modelo
feature_extractor = models.resnet50(pretrained=True)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Quitar capa final
feature_extractor = feature_extractor.cuda()

dec_model = DEC(feature_extractor, n_labels=14).cuda()
optimizer = torch.optim.Adam(dec_model.parameters(), lr=1e-4)

# Entrenamiento
train_dec_multilabel(train_loader, dec_model, optimizer, n_epochs=20, output_dir="output_multilabel")

# Evaluación
metrics = evaluate_dec_multilabel(dec_model, test_loader)
print("Metrics on Test Set:", metrics)


RuntimeError: X1 and X2 must have the same number of columns. X1: 1 X2: 2048